# Building a `latincorpus` object

## Contents of this notebook

This notebook shows how to build a parsed Latin corpus (a `latincorpus` object) for OCRE texts, and how to go from a single surface form to a morphologically sensitive full-corpus search.


## Organization of this notebook

This notebook uses Scala with the almond kernel (<https://almond.sh/>).  The cells labelled "Notebook configuration" configure the almond kernel to find and import a series of custom libraries using syntax specific to the ammonite shell that almond use.  This is analogous to defining imports in a `build.sbt` file if you were using `sbt` to run scala.

The following section (labelled  "Analyis with generic Scala") consists of completely generic scala that could be used in any environment with access to the repositories and libraries configured in the section labelled "Notebook configuration".


## Notebook configuration

Set up notebook for access to libraries.  For reasons I don't understand (but perhaps having to do with asynchronous loading) I have to separate out the two steps of adding a maven repository and using `$ivy` imports with those repositories into separate notebook cells.

In [ ]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

In [ ]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:0.4.0`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::midvalidator:9.1.0`


import $ivy.`edu.holycross.shot::latphone:2.7.2`

import $ivy.`edu.holycross.shot::latincorpus:2.2.1`


## Analyis with generic Scala

Import libraries, as always in Scala:

In [ ]:
import edu.holycross.shot.nomisma._
import edu.holycross.shot.histoutils._

import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._

import edu.holycross.shot.mid.validator._


import edu.holycross.shot.latin._
import edu.holycross.shot.latincorpus._

import scala.io.Source
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


### Build a `latincorpus`

In [ ]:
val fstUrl = "https://raw.githubusercontent.com/neelsmith/hctexts/master/workfiles/ocre/ocre-fst.txt"
val fstLines = Source.fromURL(fstUrl).getLines.toVector

val url = "https://raw.githubusercontent.com/neelsmith/hctexts/master/cex/ocre43k.cex"
val ctsLines = Source.fromURL(url).getLines.toVector.tail.filter(_.nonEmpty)

val stringPairs = ctsLines.map(_.split("#"))
val citableNodes = stringPairs.map( arr => CitableNode(CtsUrn(arr(0)), arr(1)))
val corpus = Corpus(citableNodes)


In [ ]:
val ocrelat = LatinCorpus.fromFstLines(corpus, Latin24Alphabet, fstLines, strict = false)


## `Libertas` in Ocre


In [ ]:
val lexemeIds = ocrelat.tokenLexemeIndex("libertas")
val lexemeId = lexemeIds(0)
val occurrences =  ocrelat.lexemeConcordance(lexemeId)


Convert text references to coin IDs


In [ ]:
val coinIds = occurrences.map(_.collapsePassageTo(3).passageComponent)